In [ ]:
from operators import BlockMatrixOperator, BlockTriangularPreconditioner
from tunnel_with_subdomains import load_matrices_from_hdf5, load_vectors_from_hdf5

problem_no = 19
path_to_data = f"exported_data/testrun_{problem_no}/"

# loading
A_blocks = load_matrices_from_hdf5(f"{path_to_data}data_as_blocks.h5")
Ptdiagblocks = load_matrices_from_hdf5(f"{path_to_data}data_diag_preco.h5")
Ptblocks = load_matrices_from_hdf5(f"{path_to_data}data_triang_preco.h5")
rhs_u, rhs_p = load_vectors_from_hdf5(f"{path_to_data}data_rhs.h5")  # each rhs_* is a list of ndarrays
sol_u, sol_p = load_vectors_from_hdf5(f"{path_to_data}data_sol.h5")  # each rhs_* is a list of ndarrays

In [ ]:
import numpy as np
from numpy.linalg import norm
from scipy.sparse.linalg import gmres, LinearOperator

A_mat = BlockMatrixOperator(A_blocks)
Pt_inv = BlockTriangularPreconditioner(Ptblocks, False)
# Pt_inv = BlockTriangularPreconditioner(Ptdiagblocks, False)

# Define the preconditioned operator
A = LinearOperator(A_mat.shape, matvec=A_mat)
M = LinearOperator(Pt_inv.shape, matvec=Pt_inv)

In [ ]:
iter_list = []
res_list = []
true_res_list = []
err_list = []
Z = None
Z_list = []
x_sol = None

warm_up = 10
maxit = 8

for index in range(10):  # max is 716
    rhs_vec = np.concatenate([rhs_u[index], rhs_p[index]])
    sol_vec = np.concatenate([sol_u[index], sol_p[index]])

    # Define a callback function to store residuals
    residuals = []
    def callback(resid):
        residuals.append(resid)

    if index > 0 and res_list[-1] < 1e-10:
        maxit -= 1
    # Solve the system using GMRES
    x_sol, info = gmres(A, rhs_vec, M=M, rtol=1e-16, maxiter=maxit, callback=callback, x0=x_sol)

    # Extract number of iterations (info > 0 indicates it stopped at maxit)
    num_iterations = len(residuals)

    iter_list.append(num_iterations)
    res_list.append(norm(M(A(x_sol) - rhs_vec)) / norm(rhs_vec))
    true_res_list.append(norm(M(A(sol_vec) - rhs_vec)) / norm(rhs_vec))
    err_list.append(norm(x_sol - sol_vec) / norm(sol_vec))
    print(f"Index {index}: {iter_list[-1]}, {res_list[-1]}, {err_list[-1]} {true_res_list[-1]}")

In [ ]:
import matplotlib.pyplot as plt
plt.semilogy(residuals)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(iter_list)
plt.show()

plt.figure()
plt.semilogy(res_list)
plt.semilogy(true_res_list)
plt.show()

plt.figure()
plt.semilogy(err_list)
plt.show()